In [128]:
import matplotlib.pyplot as plt
import librosa
import numpy as np
import pandas as pd
import os
import IPython.display as ipd
from IPython.display import clear_output
%matplotlib inline

In [129]:
root_path="C:/Users/sarib/Desktop/Gohm/SpeechRecognition/database"

In [130]:
def mfcc_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis=0)
    
    return mfccs_scaled_features

In [131]:
#NOT USED
def fft_extractor(file):
    signal, sample_rate = librosa.load(file)
    ipd.Audio(file)
    fft = np.fft.fft(signal)
    
    return fft

In [132]:
#NOT USED
signal, sample_rate = librosa.load('database/ac/ac_AHCG_FSRPNCW.wav')
ipd.Audio('database/ac/ac_AHCG_FSRPNCW.wav')
fft = np.fft.fft(signal)
fft

array([ 3.96959355+0.j        , -1.57368256-0.43014124j,
       -0.5562213 +0.51962332j, ...,  0.00995858+0.12657352j,
       -0.5562213 -0.51962332j, -1.57368256+0.43014124j])

In [133]:
#NOT USED
signal, sample_rate = librosa.load('database\sol\sol_AKZI_FAWCVUT.wav' )
ipd.Audio('database\sol\sol_AKZI_FAWCVUT.wav' )
mfccs_features = librosa.feature.mfcc(y=signal, sr=sample_rate, n_mfcc=40)
mfccs_features.shape

(40, 44)

In [134]:
"""from tqdm import tqdm
import time"""
extracted_feataures =[]
for root, directories, files in os.walk(root_path):
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names))
            data = mfcc_extractor(os.path.join(root,file_names))
            extracted_feataures.append([data,os.path.basename(os.path.normpath(root))])

    """for i in tqdm (range (100), desc="Loading..."):
        time.sleep(3)"""

In [135]:
extracted_feataures_dataframe= pd.DataFrame(extracted_feataures, columns=["mfcc", "word"])
extracted_feataures_dataframe.head()

,mfcc,word
0,"[-483.0338, 62.193104, -10.424236, 17.89255, -...",ac
1,"[-393.0876, 61.467396, -20.944004, 18.326431, ...",ac
2,"[-557.86304, 99.75818, -6.094904, 0.65864086, ...",ac
3,"[-518.9073, 52.873825, -3.625966, 5.807297, -9...",ac
4,"[-539.5348, 84.21058, -6.7271714, 4.8634925, -...",ac


In [136]:
X=np.array(extracted_feataures_dataframe['mfcc'].tolist())
Y=np.array(extracted_feataures_dataframe['word'].tolist())

In [137]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
Y_numerical=to_categorical(labelencoder.fit_transform(Y))

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y_numerical, test_size=0.2, random_state=0)

In [139]:
#NOT USED
X_train.shape

(21153, 40)

In [140]:
#NOT USED
len(Y_test)

5289

ANN Model

In [141]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [166]:
model= Sequential()

model.add(Flatten(input_shape=(40, )))
model.add(Dense(300, Activation('relu')))
model.add(Dropout(0.2))
model.add(Dense(300, Activation('relu')))
model.add(Dense(Y_numerical.shape[1], Activation('softmax')))

"""
model.add(Dense(200, input_shape=(40,)))
model.add(Activation('relu'))

model.add(Dense(300))
model.add(Activation('relu'))

model.add(Dense(50))
model.add(Activation('relu'))

model.add(Dense(Y_numerical.shape[1]))
model.add(Activation('sigmoid'))"""

"\nmodel.add(Dense(200, input_shape=(40,)))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(300))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(50))\nmodel.add(Activation('relu'))\n\nmodel.add(Dense(Y_numerical.shape[1]))\nmodel.add(Activation('sigmoid'))"

In [167]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_6 (Flatten)         (None, 40)                0         
                                                                 
 dense_22 (Dense)            (None, 300)               12300     
                                                                 
 dropout_4 (Dropout)         (None, 300)               0         
                                                                 
 dense_23 (Dense)            (None, 300)               90300     
                                                                 
 dense_24 (Dense)            (None, 14)                4214      
                                                                 
Total params: 106,814
Trainable params: 106,814
Non-trainable params: 0
_________________________________________________________________


In [168]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer=Adam())

In [169]:
#checkpointer = ModelCheckpoint(filepath='audio_classification.hdf5', 
 #                              verbose=0, save_best_only=True)

#model.fit(X_train, Y_train, epochs=70, validation_data=(X_test, Y_test), callbacks=[checkpointer], verbose=2)
model.fit(X_train, Y_train, epochs=70, validation_data=(X_test, Y_test), verbose=2)



Epoch 1/70
662/662 - 2s - loss: 4.2873 - accuracy: 0.2474 - val_loss: 1.6349 - val_accuracy: 0.4458 - 2s/epoch - 4ms/step
Epoch 2/70
662/662 - 2s - loss: 1.7192 - accuracy: 0.4097 - val_loss: 1.4826 - val_accuracy: 0.4942 - 2s/epoch - 3ms/step
Epoch 3/70
662/662 - 2s - loss: 1.5274 - accuracy: 0.4664 - val_loss: 1.3585 - val_accuracy: 0.5315 - 2s/epoch - 3ms/step
Epoch 4/70
662/662 - 2s - loss: 1.4195 - accuracy: 0.5062 - val_loss: 1.2831 - val_accuracy: 0.5540 - 2s/epoch - 3ms/step
Epoch 5/70
662/662 - 2s - loss: 1.3314 - accuracy: 0.5386 - val_loss: 1.1554 - val_accuracy: 0.6071 - 2s/epoch - 3ms/step
Epoch 6/70
662/662 - 2s - loss: 1.2704 - accuracy: 0.5579 - val_loss: 1.1482 - val_accuracy: 0.6145 - 2s/epoch - 3ms/step
Epoch 7/70
662/662 - 2s - loss: 1.2082 - accuracy: 0.5774 - val_loss: 1.0815 - val_accuracy: 0.6355 - 2s/epoch - 3ms/step
Epoch 8/70
662/662 - 2s - loss: 1.1571 - accuracy: 0.5933 - val_loss: 1.1105 - val_accuracy: 0.6122 - 2s/epoch - 3ms/step
Epoch 9/70
662/662 - 2s 

In [170]:
print("Test Score: ", model.evaluate(X_test,Y_test))
print("Train Score: ", model.evaluate(X_train,Y_train))

166/166 [==============================] - 0s 2ms/step - loss: 0.8489 - accuracy: 0.7669
Test Score:  [0.8488948345184326, 0.7668746709823608]
662/662 [==============================] - 1s 2ms/step - loss: 0.2014 - accuracy: 0.9327
Train Score:  [0.20143595337867737, 0.9327282309532166]


PREDICTION WITH DATA

In [171]:
predictiondata_feataures =[]
counter_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/RecordsFromDataset"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicted_label=np.argmax(model.predict(rl), axis=-1)
            prediction_class = labelencoder.inverse_transform(predicted_label) 
            predictiondata_feataures.append([os.path.basename(os.path.normpath(root)), prediction_class, predicted_label])
            if os.path.basename(os.path.normpath(root))==prediction_class:
                counter_true+=1

In [172]:
predictiondata_feataures_dataframe= pd.DataFrame(predictiondata_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentage= (counter_true/len(predictiondata_feataures))*100
print("The percantage of the true predicted data: ", percentage,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondata_feataures_dataframe


The percantage of the true predicted data:  69.04761904761905 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[sag],[11]
2,ac,[ac],[0]
3,asagi,[yukari],[13]
4,asagi,[evet],[5]
5,asagi,[hayir],[7]
6,baslat,[baslat],[2]
7,baslat,[baslat],[2]
8,baslat,[ac],[0]
9,devam,[devam],[3]


In [173]:
predictiondatas_feataures =[]
counters_true=0
for root, directories, files in os.walk("C:/Users/sarib/Desktop/Gohm/SpeechRecognition/Records"): 
    for file_names in files:
        if root is not root_path:
            #print(os.path.join(root,file_names) )
            r = mfcc_extractor(os.path.join(root,file_names))
            rl=r.reshape(1,-1)
            predicteds_label=np.argmax(model.predict(rl), axis=-1)
            predictions_class = labelencoder.inverse_transform(predicteds_label) 
            predictiondatas_feataures.append([os.path.basename(os.path.normpath(root)), predictions_class, predicteds_label])
            if os.path.basename(os.path.normpath(root))==predictions_class:
                counters_true+=1

In [174]:
predictiondatas_feataures_dataframe= pd.DataFrame(predictiondatas_feataures, columns=["actual data", "predicted data", "predicted data code"])
percentages= (counters_true/len(predictiondatas_feataures))*100
print("The percantage of the true predicted data: ", percentages,"%")
#predictiondata_feataures_dataframe.to_csv("output.csv")
predictiondatas_feataures_dataframe


The percantage of the true predicted data:  54.761904761904766 %


,actual data,predicted data,predicted data code
0,ac,[ac],[0]
1,ac,[ac],[0]
2,ac,[ac],[0]
3,asagi,[asagi],[1]
4,asagi,[asagi],[1]
5,asagi,[asagi],[1]
6,baslat,[baslat],[2]
7,baslat,[sag],[11]
8,baslat,[baslat],[2]
9,devam,[sag],[11]


In [175]:
"""filename="predictionData/dursultan.wav"
r=mfcc_extractor(filename)
print(r)
rl=r.reshape(1,-1)
print(rl)
print(rl.shape)
predicted_label=np.argmax(model.predict(rl), axis=-1)

print(predicted_label)
prediction_class = labelencoder.inverse_transform(predicted_label) 
prediction_class"""

'filename="predictionData/dursultan.wav"\nr=mfcc_extractor(filename)\nprint(r)\nrl=r.reshape(1,-1)\nprint(rl)\nprint(rl.shape)\npredicted_label=np.argmax(model.predict(rl), axis=-1)\n\nprint(predicted_label)\nprediction_class = labelencoder.inverse_transform(predicted_label) \nprediction_class'